In [98]:
# Loading required libraries
import csv
import sys
import pandas as pd
import os
import glob
import itertools
import numpy as np
import copy
import math
import time
from pathlib import Path
from collections import Counter

# Loading trace : Needs to expanded into 4K chunks
path = r'C:\Users\cchak\Desktop\Data_ECML\VDI Traces\selected_few'
all_files = glob.glob(os.path.join(path, "synthetic_dataprep_B_with_deathtime_new_1GB_SSD.csv"))

f = all_files[0]  # Change the file name as required
print("Working with file " + str(f))
cols = ['IO_Num','LBA','Deathtime_RWI']
df = pd.read_csv(f,engine='python',skiprows =1,header=None,na_values=['-1'], index_col=False)
df.columns = cols
df['Deathtime_RWI'] = df['Deathtime_RWI'].replace(np.NaN, -1)
lba_list = df['LBA'].tolist()
print("Min LBA in the dataset :", min(lba_list))
print("Max LBA in the dataset :", max(lba_list))
print("Number of unique LBAs in the data :",len(Counter(df['LBA'])))
print("Number of IO Accesses :",len(df))

Working with file C:\Users\cchak\Desktop\Data_ECML\VDI Traces\selected_few\synthetic_dataprep_B_with_deathtime_new_1GB_SSD.csv
Min LBA in the dataset : 0
Max LBA in the dataset : 262144
Number of unique LBAs in the data : 262145
Number of IO Accesses : 1310725


In [117]:
# SSD specifications
page_size = 4096
page_per_block = 64                          # Hyperparameter 
over_provisioning_ratio = 0.3 
GB = 1024*1024*1024
SSD_size_GB = 1.349 * GB
ssd_capacity = SSD_size_GB                    # Hyperparameter                     
LOG_PAGE_PER_BLOCK = int(math.log(page_per_block,2))



# Make the block,page and physical addresses for normal and Overprovisioned capacity
page_addresses = []
block_addresses = []
block_placement = 0
start_counter = -1
block_addresses.append(0)

while(start_counter < (ssd_capacity/page_size) - page_size):
    start_counter = start_counter + 1
    page_addresses.append(int(start_counter))
    if(block_placement >= page_per_block):
        block_addresses.append(int(start_counter))
        block_placement = 0

    block_placement = block_placement + 1

free_list_block = copy.deepcopy(block_addresses)
free_list_page = copy.deepcopy(page_addresses)

# Defining block_structure
valid_bitmap = []
write_ptr=0
invalid_pages=0
block_struct = {}

for x in free_list_block:
    start_lba = x
    valid_bitmap = []
    for x in range(page_per_block):
        valid_bitmap.append(False)
    segment = [start_lba,invalid_pages,valid_bitmap,write_ptr]
    block_struct[start_lba]=segment
    
GC_THRESHOLD = int(0.25*len(block_addresses))
if(len(page_addresses)*0.75 < len(Counter(df['LBA']))):
    print("WARNING...! Not enough blocks. Need to increase SSD Size")
else:
    print('Difference',(len(page_addresses)*0.75) - (len(Counter(df['LBA']))))
    print("Looks good.Go ahead!")

Difference 8.5
Looks good.Go ahead!


In [118]:
def invalidate_lba(lba):
    prev = L2P[lba]
    prev_block = (prev >> LOG_PAGE_PER_BLOCK)*page_per_block
    prev_page = prev % page_per_block
    block_details = block_struct[prev_block]                                 # Getting block details
    block_struct[prev_block][2][prev_page] = False                           # Setting bitmap to False
    block_struct[prev_block][1] = block_struct[cur_block][2].count(True)     # Setting invalid pages
    L2P.pop(lba)
    

def map_lba(lba,cur_block):
    phys_addr = block_struct[cur_block][0] + block_struct[cur_block][3]
    L2P[lba] = phys_addr
    P2L[phys_addr] = lba   
    block_struct[cur_block][2][block_struct[cur_block][3]] =  True               # Setting Bitmap
    block_struct[cur_block][1] =  block_struct[cur_block][2].count(True)         # Setting invalid pages
    block_struct[cur_block][3] = block_struct[cur_block][3] + 1                  # Increasing Write pointer
    
def check_GC (cur_block, in_gc):
    if (block_struct[cur_block][3] < page_per_block):
        return cur_block
    else:
        closed_blocks.append(cur_block)
        if(len(free_list_block) <= 0):
            print("FAIL WHILE DOING GC, RAN OUT OF BLOCKS") 
        elif (len(free_list_block) <= GC_THRESHOLD):
            if(in_gc != True):
                if(block_struct[cur_block][3] >= page_per_block):
                    cur_block = free_list_block.pop(0)
                in_gc = do_greedy_gc(cur_block,in_gc)
        if(block_struct[cur_block][3] == (page_per_block)):
            cur_block = free_list_block.pop(0)
        return cur_block

def do_greedy_gc(cur_block,in_gc):
    in_gc = True
    gc_writes = 0
    page_per_block = 64
    min_val = float('inf')  
    for x in closed_blocks:               
        if (block_struct[x][1] < min_val):
            min_val = block_struct[x][1]
            gc_blk = x

    
    #found the block with minimal valid pages, move all valid pages
    for pg in range(page_per_block):
        #figure out the logical addresses for all phys pages in the gc block
        phys_addr = block_struct[gc_blk][0] + pg
        gc_lba = P2L[phys_addr]
        P2L.pop(gc_lba)
        
        # Checking for valid bitmap
        prev_block = (phys_addr >> LOG_PAGE_PER_BLOCK)*page_per_block
        prev_page = phys_addr % page_per_block
        bitmap = block_struct[prev_block][2][prev_page]
        # If valid bitmap is True (data is valid), copy to OP capacity, increase GC writes
        if (bitmap == True):
            invalidate_lba(gc_lba)
            #check if we need to get a new block
            cur_block = check_GC(cur_block,in_gc)
            #move the gc'ed block t-o a new location
            map_lba(gc_lba,cur_block)   
            gc_writes = gc_writes + 1
           
            
    if(gc_writes > 64):
        print("GC writes not as expected", gc_writes)
        
    # Reset block details, remove from closed list and add to free_list
    invalid_pages = 0
    valid_bitmap = []
    write_ptr = 0
    for x in range(page_per_block):
        valid_bitmap.append(False)
        
    block_struct[gc_blk]= [gc_blk,invalid_pages,valid_bitmap,write_ptr]
    closed_blocks.remove(gc_blk)
    free_list_block.append(gc_blk)
    total_gc_writes[0] = total_gc_writes[0] + gc_writes
    in_gc = False
    return in_gc
    

In [119]:
# Core simulation of trace
L2P = {}
P2L = {}
total_gc_writes = []
total_gc_writes.append(0)
counter = 0
GC_THRESHOLD = int(0.25*len(block_addresses))
print("GC Threshold set",GC_THRESHOLD)
min_LBA = min(lba_list)
closed_blocks = []
cur_block = free_list_block.pop(0)
global in_gc 
in_gc = False
global gc_writes
gc_writes = 0


start_time = time.time()
while(counter < len(lba_list)):
    lba=int(lba_list[counter]) - min_LBA
    if(counter >50000 and counter%50000==0):
        print("Percentage completed in (%)  :", (counter/len(lba_list))*100)
    if lba in L2P:
        invalidate_lba(lba)
    cur_block = check_GC(cur_block,in_gc)
    map_lba(lba,cur_block)
    counter = counter + 1
    
    
print("End of Trace")
end_time = time.time()
run_time = end_time - start_time
print("Execution Time for the FTL :",run_time)
print("Total Number of GC writes :",total_gc_writes[0])


GC Threshold set 1365
Percentage completed in (%)  : 7.629365427530566
Percentage completed in (%)  : 11.444048141295848
Percentage completed in (%)  : 15.258730855061131
Percentage completed in (%)  : 19.073413568826414
Percentage completed in (%)  : 22.888096282591697
Percentage completed in (%)  : 26.70277899635698
Percentage completed in (%)  : 30.517461710122262
Percentage completed in (%)  : 34.33214442388754
Percentage completed in (%)  : 38.14682713765283
Percentage completed in (%)  : 41.961509851418114
Percentage completed in (%)  : 45.77619256518339
Percentage completed in (%)  : 49.59087527894867
Percentage completed in (%)  : 53.40555799271396
Percentage completed in (%)  : 57.220240706479245
Percentage completed in (%)  : 61.034923420244525
Percentage completed in (%)  : 64.8496061340098
Percentage completed in (%)  : 68.66428884777508
Percentage completed in (%)  : 72.47897156154038
Percentage completed in (%)  : 76.29365427530566
Percentage completed in (%)  : 80.108336